In [ ]:
import sqlite3
 
from pathlib import Path

BD = Path(__file__).parent / "productos.db"

def obtener_conexion():
    conexion = sqlite3.connect(BD)
    conexion.row_factory = sqlite3.Row
    return conexion

def inicializar_bd():
    conexion = obtener_conexion()
    conexion.execute("""
    CREATE TABLE IF NOT EXISTS productos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sku TEXT UNIQUE NOT NULL,
        nombre TEXT NOT NULL,
        precio_centavos INTEGER NOT NULL
    )
    """)
        # Productos de ejemplo
    productos = [
        ("HIELO-CUB-SM","Cubitos pequeños",500),
        ("HIELO-CUB-LG","Cubitos grandes",800),
        ("HIELO-TRI","Hielo triturado",600)
    ]
    for sku, nombre, precio in productos:
        cur = conexion.execute("SELECT * FROM productos WHERE sku=?", (sku,))
        if not cur.fetchone():
            conexion.execute("INSERT INTO productos (sku,nombre,precio_centavos) VALUES (?,?,?)",
                             (sku,nombre,precio))
    conexion.commit()
    conexion.close() 

In [ ]:
from flask import Flask
from bd import inicializar_bd
from rutas import registrar_rutas

def crear_app():
    inicializar_bd()
    app = Flask(__name__)
    registrar_rutas(app)
    return app

if __name__=="__main__":
    crear_app().run(port=5001, debug=True)

In [ ]:
from flask import request, jsonify
from bd import obtener_conexion

TOKEN_SECRETO = "token_mvp_hielos"

def verificar_token(token):
    return token == TOKEN_SECRETO

def registrar_rutas(app):
    @app.get("/productos")
    def listar_productos():
        conexion = obtener_conexion()
        filas = conexion.execute("SELECT * FROM productos").fetchall()
        conexion.close()
        return jsonify([dict(f) for f in filas])

    @app.post("/productos")
    def crear_producto():
        auth = request.headers.get("Authorization","")
        if not auth.startswith("Bearer ") or not verificar_token(auth.split(" ")[1]):
            return jsonify({"error":"token invalido"}), 401
        datos = request.json or {}
        if not all(k in datos for k in ("sku","nombre","precio_centavos")):
            return jsonify({"error":"faltan campos"}), 400
        conexion = obtener_conexion()
        try:
            conexion.execute("INSERT INTO productos (sku,nombre,precio_centavos) VALUES (?,?,?)",
                             (datos["sku"],datos["nombre"],datos["precio_centavos"]))
            conexion.commit()
            conexion.close()
            return jsonify({"mensaje":"creado"}), 201
        except Exception as e:
            conexion.close()
            return jsonify({"error": str(e)}), 400

In [ ]:
import sqlite3
from pathlib import Path

BD = Path(__file__).parent / "inventario.db"

def obtener_conexion():
    conexion = sqlite3.connect(BD)
    conexion.row_factory = sqlite3.Row
    return conexion

def inicializar_bd():
    conexion = obtener_conexion()
    conexion.execute("""
    CREATE TABLE IF NOT EXISTS inventario (
        sku TEXT PRIMARY KEY,
        disponible INTEGER NOT NULL
    )
    """)
    inventario_inicial = [
        ("HIELO-CUB-SM",50),
        ("HIELO-CUB-LG",30),
        ("HIELO-TRI",40)
    ]
    for sku,cant in inventario_inicial:
        cur = conexion.execute("SELECT * FROM inventario WHERE sku=?", (sku,))
        if not cur.fetchone():
            conexion.execute("INSERT INTO inventario (sku,disponible) VALUES (?,?)",(sku,cant))
    conexion.commit()
    conexion.close()

In [ ]:
from flask import Flask
from bd import inicializar_bd
from rutas import registrar_rutas


def crear_app():
    inicializar_bd()
    app = Flask(__name__)
    registrar_rutas(app)
    return app

if __name__=="__main__":
    crear_app().run(port=5002, debug=True)


In [ ]:
from flask import request, jsonify
from bd import obtener_conexion

TOKEN_SECRETO = "token_mvp_hielos"

def verificar_token(token):
    return token == TOKEN_SECRETO

def registrar_rutas(app):
    @app.get("/inventario/<string:sku>")
    def consultar_stock(sku):
        conexion = obtener_conexion()
        fila = conexion.execute("SELECT * FROM inventario WHERE sku=?", (sku,)).fetchone()
        conexion.close()
        if not fila:
            return jsonify({"sku":sku,"disponible":0})
        return jsonify(dict(fila))

    @app.post("/inventario/reservar")
    def reservar_stock():
        auth = request.headers.get("Authorization","")
        if not auth.startswith("Bearer ") or not verificar_token(auth.split(" ")[1]):
            return jsonify({"error":"token invalido"}), 401
        datos = request.json or {}
        sku = datos.get("sku")
        cantidad = int(datos.get("cantidad",0))
        if not sku or cantidad <= 0:
            return jsonify({"error":"datos invalidos"}),400
        conexion = obtener_conexion()
        fila = conexion.execute("SELECT disponible FROM inventario WHERE sku=?", (sku,)).fetchone()
        if not fila or fila["disponible"]<cantidad:
            conexion.close()
            return jsonify({"error":"stock insuficiente"}),409
        conexion.execute("UPDATE inventario SET disponible=disponible-? WHERE sku=?",(cantidad,sku))
        conexion.commit()
        conexion.close()
        return jsonify({"mensaje":"reservado","sku":sku,"cantidad":cantidad})

    @app.post("/inventario/liberar")
    def liberar_stock():
        auth = request.headers.get("Authorization","")
        if not auth.startswith("Bearer ") or not verificar_token(auth.split(" ")[1]):
            return jsonify({"error":"token invalido"}), 401
        datos = request.json or {}
        sku = datos.get("sku")
        cantidad = int(datos.get("cantidad",0))
        if not sku or cantidad <= 0:
            return jsonify({"error":"datos invalidos"}),400
        conexion = obtener_conexion()
        fila = conexion.execute("SELECT disponible FROM inventario WHERE sku=?", (sku,)).fetchone()
        if not fila:
            conexion.execute("INSERT INTO inventario(sku,disponible) VALUES (?,?)",(sku,cantidad))
        else:
            conexion.execute("UPDATE inventario SET disponible=disponible+? WHERE sku=?",(cantidad,sku))
        conexion.commit()
        conexion.close()
        return jsonify({"mensaje":"liberado","sku":sku,"cantidad":cantidad})

In [ ]:
import sqlite3
from pathlib import Path

BD = Path(__file__).parent / "pagos.db"

def obtener_conexion():
    conexion = sqlite3.connect(BD)
    conexion.row_factory = sqlite3.Row
    return conexion

def inicializar_bd():
    conexion = obtener_conexion()
    conexion.execute("""
    CREATE TABLE IF NOT EXISTS pagos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        monto_centavos INTEGER NOT NULL,
        metodo TEXT NOT NULL,
        estado TEXT NOT NULL
    )
    """)
    conexion.commit()
    conexion.close()


In [ ]:
from flask import Flask
from bd import inicializar_bd
from rutas import registrar_rutas

def crear_app():
    inicializar_bd()
    app = Flask(__name__)
    registrar_rutas(app)
    return app

if __name__=="__main__":
    crear_app().run(port=5003, debug=True)


In [ ]:
from flask import request, jsonify
from bd import obtener_conexion

TOKEN_SECRETO = "token_mvp_hielos"

def verificar_token(token):
    return token == TOKEN_SECRETO

def registrar_rutas(app):
    @app.post("/pago")
    def realizar_pago():
        auth = request.headers.get("Authorization","")
        if not auth.startswith("Bearer ") or not verificar_token(auth.split(" ")[1]):
            return jsonify({"error":"token invalido"}),401
        datos = request.json or {}
        monto = int(datos.get("monto_centavos",0))
        metodo = datos.get("metodo","desconocido")
        if monto <=0:
            return jsonify({"error":"monto invalido"}),400
        conexion = obtener_conexion()
        conexion.execute("INSERT INTO pagos(monto_centavos,metodo,estado) VALUES (?,?,?)",(monto,metodo,"completado"))
        conexion.commit()
        conexion.close()
        return jsonify({"mensaje":"pago realizado","monto_centavos":monto,"metodo":metodo})


In [ ]:
import requests

TOKEN = "token_mvp_hielos"

URL_PRODUCTOS = "http://127.0.0.1:5001/productos"
URL_INVENTARIO = "http://127.0.0.1:5002/inventario"
URL_PAGO = "http://127.0.0.1:5003/pago"

def mostrar_productos():
    r = requests.get(URL_PRODUCTOS)
    for p in r.json():
        print(f"{p['sku']}: {p['nombre']} - ${p['precio_centavos']/100:.2f}")

def consultar_stock():
    sku = input("SKU del producto: ")
    r = requests.get(f"{URL_INVENTARIO}/{sku}")
    print(f"Stock disponible: {r.json().get('disponible',0)}")

def reservar_hielo():
    sku = input("SKU del producto: ")
    cantidad = int(input("Cantidad: "))
    r = requests.post(f"{URL_INVENTARIO}/reservar",
                      headers={"Authorization":f"Bearer {TOKEN}"},
                      json={"sku":sku,"cantidad":cantidad})
    print(r.json())

def liberar_hielo():
    sku = input("SKU del producto: ")
    cantidad = int(input("Cantidad: "))
    r = requests.post(f"{URL_INVENTARIO}/liberar",
                      headers={"Authorization":f"Bearer {TOKEN}"},
                      json={"sku":sku,"cantidad":cantidad})
    print(r.json())

def pagar():
    monto = int(input("Monto en centavos: "))
    metodo = input("Método de pago: ")
    r = requests.post(f"{URL_PAGO}",
                      headers={"Authorization":f"Bearer {TOKEN}"},
                      json={"monto_centavos":monto,"metodo":metodo})
    print(r.json())

def menu():
    while True:
        print("\nTienda de Hielos")
        print("1) Listar productos")
        print("2) Consultar stock")
        print("3) Reservar hielo")
        print("4) Liberar hielo")
        print("5) Pagar")
        print("6) Salir")
        opcion = input("Opción: ")
        if opcion=="1": mostrar_productos()
        elif opcion=="2": consultar_stock()
        elif opcion=="3": reservar_hielo()
        elif opcion=="4": liberar_hielo()
        elif opcion=="5": pagar()
        elif opcion=="6": break
        else: print("Opción inválida")

if __name__=="__main__":
    menu()
